In [ ]:
#Cargamos las librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
#Cargamos el dataset con el que vamos a trabajar y le damos un vistazo
df = pd.read_csv("https://github.com/LHukovsky/Coderhouse-Final-Project-/raw/"+
                 "main/encuesta-anual-hogares-2019.csv",
                 encoding="latin1")
df.head()

,id,nhogar,miembro,comuna,dominio,edad,sexo,parentesco_jefe,situacion_conyugal,num_miembro_padre,...,ingreso_per_capita_familiar,estado_educativo,sector_educativo,nivel_actual,nivel_max_educativo,años_escolaridad,lugar_nacimiento,afiliacion_salud,hijos_nacidos_vivos,cantidad_hijos_nac_vivos
0,1,1,1,5,Resto de la Ciudad,18,Mujer,Jefe,Soltero/a,Padre no vive en el hogar,...,9000,Asiste,Estatal/publico,Universitario,Otras escuelas especiales,12,PBA excepto GBA,Solo obra social,No,No corresponde
1,1,1,2,5,Resto de la Ciudad,18,Mujer,Otro no familiar,Soltero/a,Padre no vive en el hogar,...,9000,Asiste,Estatal/publico,Universitario,Otras escuelas especiales,12,Otra provincia,Solo plan de medicina prepaga por contratación...,No,No corresponde
2,2,1,1,2,Resto de la Ciudad,18,Varon,Jefe,Soltero/a,Padre no vive en el hogar,...,33333,Asiste,Privado religioso,Universitario,Otras escuelas especiales,12,CABA,Solo plan de medicina prepaga por contratación...,NaN,No corresponde
3,2,1,2,2,Resto de la Ciudad,50,Mujer,Padre/Madre/Suegro/a,Viudo/a,No corresponde,...,33333,No asiste pero asistió,No corresponde,No corresponde,Secundario/medio comun,17,CABA,Solo prepaga o mutual via OS,Si,2
4,2,1,3,2,Resto de la Ciudad,17,Varon,Otro familiar,Soltero/a,Padre no vive en el hogar,...,33333,Asiste,Privado religioso,Secundario/medio comun,EGB (1° a 9° año),10,CABA,Solo plan de medicina prepaga por contratación...,NaN,No corresponde


# Transformación de la base

Generamos diversas transformaciones de variables, así como la creación de la variale "Target", pues es la que usaremos para todo el análisis.

In [ ]:
 #Creamos el target para nivel_max_educativo
df['Target'] = df['nivel_max_educativo']

df.loc[df['Target'].isin(['Secundario/medio comun','EGB (1° a 9° año)']),'Target'] = 'sec_completo'
df.loc[df['Target'].isin(['Primario especial','Primario comun']),'Target']         = 'prim_completo'
df.loc[df['Target']    == 'Sala de 5','Target']                                    = 'inicial'
df.loc[df['Target']    == 'Otras escuelas especiales','Target']                    = 'superior'

#Remplazo los valores de años_escolaridad para que todos sean numéricos
df.loc[df.años_escolaridad=="Ningun año de escolaridad aprobado","años_escolaridad"]=0
df["años_escolaridad"] = df["años_escolaridad"].astype('float')

#La variable "cantidad_hijos_nac_vivos" se puede pasar a numérica si se toma "no corresponde" como NAN.
df.loc[df.cantidad_hijos_nac_vivos=="No corresponde","cantidad_hijos_nac_vivos"]="0"
df["cantidad_hijos_nac_vivos"] = df["cantidad_hijos_nac_vivos"].apply(int)

#Hay determinadas variables (comuna,id,nhogar y miembro) que están como numéricas pero deberían ser categóricas.
df[["comuna","id","nhogar","miembro"]] = df[["comuna","id","nhogar","miembro"]].applymap(str)

# Por último renombramos algunas variables para que sean más cortas
df.rename(columns = {#'dominio_Villas de emergencia': 'dominio_villas', 
                     'ingreso_per_capita_familiar': 'ing_per_cap_familiar',
                     "cantidad_hijos_nac_vivos" : "cant_hijos_nac_vivos"}, inplace = True)

#Reagrupamos la variable comuna por regiones para reducir la dimensionalidad
df['region'] = df['comuna']
df.loc[df['comuna'].isin(['12','13','14','2','15']),'region'] = 'norte'
df.loc[df['comuna'].isin(['5','3','1']),'region'] = 'centro'
df.loc[df['comuna'].isin(['8','4']),'region'] = 'sur'
df.loc[df['comuna'].isin(['6','7','9','10','11']),'region'] = 'oeste'

#Quitamos variables que no nos interesan
df.drop(['id', 'nhogar', 'miembro', 'num_miembro_padre', 'num_miembro_madre','nivel_max_educativo','parentesco_jefe',
         'calidad_ingresos_lab','calidad_ingresos_no_lab','calidad_ingresos_totales','calidad_ingresos_familiares',
         'hijos_nacidos_vivos','comuna'],axis=1,inplace=True)

In [ ]:
#Vamos a dropear los nulls en el target y los "no corresponde"
df = df.loc[df.Target.notnull(),]
df = df.loc[df.Target!='No corresponde',]
df.isnull().sum().where(lambda x : x>0).dropna().apply(int).sort_values()

situacion_conyugal     1
lugar_nacimiento       1
sector_educativo       2
afiliacion_salud       4
años_escolaridad      19
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13223 entries, 0 to 14318
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   dominio               13223 non-null  object 
 1   edad                  13223 non-null  int64  
 2   sexo                  13223 non-null  object 
 3   situacion_conyugal    13222 non-null  object 
 4   estado_ocupacional    13223 non-null  object 
 5   cat_ocupacional       13223 non-null  object 
 6   ingreso_total_lab     13223 non-null  int64  
 7   ingreso_total_no_lab  13223 non-null  int64  
 8   ingresos_totales      13223 non-null  int64  
 9   ingresos_familiares   13223 non-null  int64  
 10  ing_per_cap_familiar  13223 non-null  int64  
 11  estado_educativo      13223 non-null  object 
 12  sector_educativo      13221 non-null  object 
 13  nivel_actual          13223 non-null  object 
 14  años_escolaridad      13204 non-null  float64
 15  lugar_nacimiento   

Partimos train y test

In [ ]:
#spliteo de train y test 
df_train, df_test = train_test_split(df,test_size=0.20, random_state=42)

## Tratado de nulls

In [ ]:
df_train.isnull().sum().where(lambda x : x>0).dropna().apply(int).sort_values()

situacion_conyugal     1
sector_educativo       1
lugar_nacimiento       1
afiliacion_salud       2
años_escolaridad      13
dtype: int64

In [ ]:
#Remplazamos los nulls de los objects con la moda
for i in ['situacion_conyugal','lugar_nacimiento','sector_educativo','afiliacion_salud']:
  df_train.loc[df[i].isnull(),i] = df_train['situacion_conyugal'].mode()[0]

Ahora vamos a abordar los nulls en años de escolaridad

In [ ]:
# fill age con la mediana de la agrupacion Sexo y Class

#creo datarrame de referencia
age_ref_null = df_train.groupby(['sexo', 'region']).median()['años_escolaridad'].reset_index()

#funcion de replace en base a dataframe agrupado
def fill_na_age_grouped(df_grouped , row_replace):
  if pd.isna(row_replace['años_escolaridad']):
    return df_grouped[(df_grouped['sexo'] == row_replace['sexo']) & (df_grouped['region'] == row_replace['region'])]['años_escolaridad'].values[0]
  else:
    return row_replace['años_escolaridad']


#transformacion de train
df_train['años_escolaridad'] = df_train.apply(lambda row: fill_na_age_grouped(age_ref_null, row), axis=1)

df_train['años_escolaridad'] = df_train.años_escolaridad.apply(int)

# Comienzo de entrenamiento

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10578 entries, 1206 to 8190
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   dominio               10578 non-null  object
 1   edad                  10578 non-null  int64 
 2   sexo                  10578 non-null  object
 3   situacion_conyugal    10578 non-null  object
 4   estado_ocupacional    10578 non-null  object
 5   cat_ocupacional       10578 non-null  object
 6   ingreso_total_lab     10578 non-null  int64 
 7   ingreso_total_no_lab  10578 non-null  int64 
 8   ingresos_totales      10578 non-null  int64 
 9   ingresos_familiares   10578 non-null  int64 
 10  ing_per_cap_familiar  10578 non-null  int64 
 11  estado_educativo      10578 non-null  object
 12  sector_educativo      10578 non-null  object
 13  nivel_actual          10578 non-null  object
 14  años_escolaridad      10578 non-null  int64 
 15  lugar_nacimiento      10578 non-nu

Pasamos las variables string a one hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#generate one hot encoder class
encoder = OneHotEncoder(handle_unknown='ignore', #ponemos ignore para poder ommitir cuando puede pasar que no exista los mismos valores 
                        sparse=False) # generarlo como matrix para poder generar las columnas adecuadas

#entrenamiento para genera el preprocesamiento one hot encoding para las variables indicadas
o_features = ['dominio','sexo','situacion_conyugal','estado_ocupacional','cat_ocupacional','estado_educativo',
              'sector_educativo','nivel_actual','lugar_nacimiento','afiliacion_salud','region','Target']

encoder.fit(df_train[o_features])

#transformo en train datasets en base a la clase generada
cat_encoding = pd.DataFrame(encoder.transform(df_train[o_features]), columns=encoder.get_feature_names(o_features))
cat_encoding.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,dominio_Resto de la Ciudad,dominio_Villas de emergencia,sexo_Mujer,sexo_Varon,situacion_conyugal_Casado/a,situacion_conyugal_Divorciado/a,situacion_conyugal_No corresponde,situacion_conyugal_Separado/a de unión o matrimonio,situacion_conyugal_Soltero/a,situacion_conyugal_Unido/a,...,afiliacion_salud_Solo prepaga o mutual via OS,afiliacion_salud_Solo sistema publico,region_centro,region_norte,region_oeste,region_sur,Target_inicial,Target_prim_completo,Target_sec_completo,Target_superior
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
#joineo de las variables que deseo con categoricas encoding
df_all_train = pd.concat([df_train, cat_encoding], axis=1)


#Borrado de columnas no necesarias 
df_all_train.drop(o_features,axis=1, inplace=True)

#train datasets final
df_all_train

,edad,ingreso_total_lab,ingreso_total_no_lab,ingresos_totales,ingresos_familiares,ing_per_cap_familiar,años_escolaridad,cant_hijos_nac_vivos,dominio_Resto de la Ciudad,dominio_Villas de emergencia,...,afiliacion_salud_Solo prepaga o mutual via OS,afiliacion_salud_Solo sistema publico,region_centro,region_norte,region_oeste,region_sur,Target_inicial,Target_prim_completo,Target_sec_completo,Target_superior
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,18.0,0.0,12000.0,12000.0,18000.0,9000.0,12.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,18.0,0.0,0.0,0.0,100000.0,33333.0,12.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,50.0,70000.0,30000.0,100000.0,100000.0,33333.0,17.0,2.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,17.0,0.0,0.0,0.0,100000.0,33333.0,10.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14311,98.0,0.0,12250.0,12250.0,12250.0,12250.0,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14313,52.0,30000.0,0.0,30000.0,80000.0,80000.0,7.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14314,99.0,0.0,24000.0,24000.0,57000.0,14250.0,5.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14316,60.0,0.0,11000.0,11000.0,57000.0,14250.0,12.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Entrenamiento de modelo

In [ ]:
#spliteo de datos en target y features for test and train

#seleccion de x values sin el target
X_train = df_all_train.loc[:,df_all_train.columns != 'Target']

#seleccion del target
y_train = df_all_train.años_escolaridad

In [ ]:
#training del modelo
from sklearn import tree

model_default = tree.DecisionTreeClassifier(random_state = 42) # con hiperparametros default
model_default.fit(X_train, y_train)

#modelo generado con algun hiperparamtros diferente
model_max_depth = tree.DecisionTreeClassifier(random_state = 42, max_depth=7) 
model_max_depth.fit(X_train, y_train)

ValueError: ignored

In [ ]:
df_all_train.isnull().sum().where(lambda x: x>0)

edad                    2928
ingreso_total_lab       2928
ingreso_total_no_lab    2928
ingresos_totales        2928
ingresos_familiares     2928
                        ... 
region_sur              2928
Target_inicial          2928
Target_prim_completo    2928
Target_sec_completo     2928
Target_superior         2928
Length: 65, dtype: int64